<a href="https://colab.research.google.com/github/yuukimotai/GA4_QABot/blob/main/GA4_QA_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import os
origin = os.listdir('kftt-data-1.0/data/orig')
token = os.listdir('kftt-data-1.0/data/tok')

In [ ]:
originList = []# origフォルダに入っていたものから抽出する
for ori in origin:
  if ori.endswith('.ja'):
    originList.append(ori)
target_file = open('kftt_origin_ja.txt', 'w')
for i in range(len(originList)):
  f = open("kftt-data-1.0/data/orig/" + originList[i], "rb")

  data = f.read()

  utf8_data = data.decode(encoding='utf-8')
  print(utf8_data)
  target_file.write(utf8_data)
  f.close()
target_file.close()

In [ ]:
with open('rakuten_book_000.txt', 'r') as f:
  origin_ja = f.readlines()
origin_ja

In [ ]:
!mecab
#export MECABRC="[インストール先]/etc/mecabrc"
#/usr/lib/aarch64-linux-gnu/mecab/dic/ipadic

In [ ]:
import MeCab
# /usr/lib/aarch64-linux-gnu/mecab/dic/ipadic
tokenizer = MeCab.Tagger('-Owakati -d "/usr/lib/aarch64-linux-gnu/mecab/dic/ipadic" -u "/usr/lib/aarch64-linux-gnu/mecab/dic/ipadic/NEologd.20200910-u.dic"')
print(tokenizer.parse('特急はくたか'))

In [ ]:
with open("rakuten_books_tokenized.txt", mode="a") as f:
    for sentence in origin_ja:
        f.write(tokenizer.parse(sentence))

In [ ]:
with open("rakuten_book_000.txt", mode="r") as f:
    lines = f.readlines()

# 5000行ごとに分割
chunk_size = 1000
chunks = [lines[i:i + chunk_size] for i in range(0, len(lines), chunk_size)]

In [ ]:
for i, chunk in enumerate(chunks):
    with open(f"rakutenbooks_chunk_{i+1}.txt", mode="w", encoding="utf-8") as f:
        f.writelines(chunk)

print(f"分割完了: {len(chunks)}個のファイルが作成されました")

In [ ]:
with open("rakuten_books_tokenized.txt", mode="r") as f:
    lines = f.readlines()

# 5000行ごとに分割
chunk_size = 5000
chunks = [lines[i:i + chunk_size] for i in range(0, len(lines), chunk_size)]

In [ ]:
for i, chunk in enumerate(chunks):
    with open(f"rakutenbookstoken_chunk_{i+1}.txt", mode="w", encoding="utf-8") as f:
        f.writelines(chunk)

print(f"分割完了: {len(chunks)}個のファイルが作成されました")

In [ ]:
!pip3 install ipadic

In [ ]:
from sklearn.model_selection import train_test_split

with open("rakutenbooks_chunk_4.txt", mode="r", encoding="utf-8") as f:
    chunk = f.read()

train, temp = train_test_split(chunk, test_size=0.2)
validation, test = train_test_split(temp, test_size=0.5)

In [2]:
from datasets import DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
from tqdm.autonotebook import tqdm as notebook_tqdm
import torch
import random

In [3]:
from datasets import load_dataset
dataset = load_dataset("tyqiangz/multilingual-sentiments", "japanese")

In [4]:
from transformers import BertJapaneseTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

In [5]:
random.seed(30)
dataset = DatasetDict({
    "train"     : dataset['train']\
        .select(random.sample(range(dataset['train']     .num_rows), k=1000)),
    "validation": dataset['validation']\
        .select(random.sample(range(dataset['validation'].num_rows), k=1000)),
    "test"      : dataset['test']\
        .select(random.sample(range(dataset['test']      .num_rows), k=1000))
})
# トークナイザのロード
model_ckpt ="cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
# 事前学習モデルのロード
model_name = "cl-tohoku/bert-base-japanese"
access_token = ""
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 3
model = (AutoModelForSequenceClassification
    .from_pretrained(model_ckpt, num_labels=num_labels)
    .to(device))

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)
dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

batch_size = 16
logging_steps = len(dataset_encoded["train"]) // batch_size
model_name = "sample-text-classification-bert"
training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    eval_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps,
    log_level="error"
)
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"],
    tokenizer=tokenizer
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Parameter 'function'=<function tokenize at 0xffff247ed550> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100% 1000/1000 [00:00<00:00, 3596.45 examples/s]
Map: 100% 1000/1000 [00:00<00:00, 4106.80 examples/s]
Map: 100% 1000/1000 [00:00<00:00, 4844.30 exam

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.749500,0.622454,0.738000,0.741682
2,0.488200,0.628679,0.749000,0.738637
3,0.234900,0.790660,0.774000,0.772901
4,0.151500,0.967295,0.762000,0.755455
5,0.074500,0.999401,0.766000,0.763516


TrainOutput(global_step=625, training_loss=0.35394152866601947, metrics={'train_runtime': 20431.3564, 'train_samples_per_second': 0.245, 'train_steps_per_second': 0.031, 'total_flos': 1315567088640000.0, 'train_loss': 0.35394152866601947, 'epoch': 5.0})

In [6]:
model.save_pretrained("cl-tohoku_classification")

In [ ]:
model_name = "cl-tohoku/bert-base-japanese"
access_token = ""
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2, token=access_token)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs"
)
with open(f"rakutenbookstoken_chunk_{1}.txt", mode="r", encoding="utf-8") as f:
    tokens =  f.readlines()
for i in range(len(chunks)):
    with open(f"rakutenbooks_chunk_{i+1}.txt", mode="r", encoding="utf-8") as f:
        lines =  f.readlines()
    inputs = tokenizer(lines, return_tensors='pt', padding=True, truncation=True)
    print(len(inputs["input_ids"]))
    # トレーニングデータセットの作成
    dataset = Dataset.from_dict({'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask'], 'labels': lines})

    trainer = Trainer(
        model = model,
        args = training_args,
        train_dataset = dataset
    )
    
    trainer.train()
    
    model.save_pretrained(f"rakutenbooks_model_chunk_{i+1}")

print("ファインチューニング完了")

In [ ]:
from transformers import BertJapaneseTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

model_name = "cl-tohoku/bert-base-japanese"
access_token = ""
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2, token=access_token)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs"
)
with open(f"rakutenbookstoken_chunk_{1}.txt", mode="r", encoding="utf-8") as f:
    tokens =  f.readlines()
for i in range(len(chunks)):
    with open(f"rakutenbooks_chunk_{i+1}.txt", mode="r", encoding="utf-8") as f:
        lines =  f.readlines()
    inputs = tokenizer(lines, return_tensors='pt', padding=True, truncation=True)
    print(len(inputs["input_ids"]))
    # トレーニングデータセットの作成
    dataset = Dataset.from_dict({'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask'], 'labels': lines})

    trainer = Trainer(
        model = model,
        args = training_args,
        train_dataset = dataset
    )
    
    trainer.train()
    
    model.save_pretrained(f"rakutenbooks_model_chunk_{i+1}")

print("ファインチューニング完了")

In [ ]:
import pickle

hiragana = "ぁあぃいぅうぇえぉおかがきぎくぐけげこごさざしじすずせぜそぞ\
ただちぢっつづてでとどなにぬねのはばぱひびぴふぶぷへべぺほぼぽ\
まみむめもゃやゅゆょよらりるれろゎわゐゑをん"

katakana = "ァアィイゥウェエォオカガキギクグケゲコゴサザシジスズセゼソゾ\
タダチヂッツヅテデトドナニヌネノハバパヒビピフブプヘベペホボポ\
マミムメモャヤュユョヨラリルレロヮワヰヱヲンヴ"

chars = hiragana + katakana

for char in origin_ja:  # ひらがな、カタカナ以外でコーパスに使われている文字を追加
    if char not in chars:
        chars += char

chars += "\t\n"  # タブと改行を追加

chars_list = sorted(list(chars))  # 文字列をリストに変換してソートする
print(chars_list)

with open("chumonno_oi_ryoriten.pickle", mode="wb") as f:  # pickleで保存
    pickle.dump(chars_list, f)

In [ ]:
import pickle

with open('chumonno_oi_ryoriten.pickle', mode='rb') as f:
    chars_list = pickle.load(f)
print(chars_list)

def is_invalid(message):
    is_invalid =False
    for char in message:
        if char not in chars_list:
            is_invalid = True
    return is_invalid

In [ ]:
import numpy as np

# インデックスと文字で辞書を作成
char_indices = {}
for i, char in enumerate(chars_list):
    char_indices[char] = i
indices_char = {}
for i, char in enumerate(chars_list):
    indices_char[i] = char

n_char = len(chars_list)
max_length_x = 128

# 文章をone-hot表現に変換する関数
# def sentence_to_vector(sentence):
#     vector = np.zeros((1, max_length_x, n_char), dtype=np.bool_)
#     for j, char in enumerate(sentence):
#         vector[0][j][char_indices[char]] = 1
#     return vector
def sentence_to_vector(sentence):
    vector = np.zeros((1, max_length_x, n_char), dtype=np.bool_)
    for j, char in enumerate(sentence):
        if char in char_indices:
            vector[0][j][char_indices[char]] = 1
        else:
            print(f"Character '{char}' not in char_indices.")
    return vector

In [ ]:
char_indices

In [ ]:
#!wget -P /content/drive/MyDrive/nlp_dataset/ http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz

In [ ]:
import tarfile

# tar.gzファイルを開く
with tarfile.open('drive/MyDrive/nlp_dataset/kftt-data-1.0.tar.gz', 'r:gz') as tar:
    tar.extractall(path="drive/MyDrive/nlp_dataset")

In [ ]:
tokenList = []# tokフォルダに入っていたものから抽出する
for tok in token:
  if tok.endswith('.ja'):
    tokenList.append(tok)
target_file = open('kftt_token_ja.txt', 'w')
for i in range(len(tokenList)):
  f = open("drive/MyDrive/nlp_dataset/kftt-data-1.0/data/tok/" + tokenList[i], "rb")

  data = f.read()

  utf8_data = data.decode(encoding='utf-8')
  print(utf8_data)
  target_file.write(utf8_data)
  f.close()
target_file.close()

In [ ]:
origin_ja

In [ ]:
import numpy as np

# インデックスと文字で辞書を作成
char_indices = {}  # 文字がキーでインデックスが値
for i, char in enumerate(chars_list):
    char_indices[char] = i
indices_char = {}  # インデックスがキーで文字が値
for i, char in enumerate(chars_list):
    indices_char[i] = char

seperator = " "
sentence_list = origin_ja.split(seperator)
print(sentence_list)
sentence_list.pop()
sentence_list = [x+seperator for x in sentence_list]

max_sentence_length = 128  # 文章の最大長さ。これより長い文章はカットされる。
sentence_list = [sentence for sentence in sentence_list if len(sentence) <= max_sentence_length]  # 長すぎる文章のカット

n_char = len(chars_list)  # 文字の種類の数
n_sample = len(sentence_list) - 1  # サンプル数

x_sentences = []  # 入力の文章
t_sentences = []  # 正解の文章
for i in range(n_sample):
    x_sentences.append(sentence_list[i])
    t_sentences.append("\t" + sentence_list[i+1] + "\n")  # 正解は先頭にタブ、末尾に改行を加える
max_length_x = max_sentence_length  # 入力文章の最大長さ
max_length_t = max_sentence_length + 2  # 正解文章の最大長さ

x_encoder = np.zeros((n_sample, max_length_x, n_char), dtype=np.bool_)  # encoderへの入力
x_decoder = np.zeros((n_sample, max_length_t, n_char), dtype=np.bool_)  # decoderへの入力
t_decoder = np.zeros((n_sample, max_length_t, n_char), dtype=np.bool_)  # decoderの正解

for i in range(n_sample):
    x_sentence = x_sentences[i]
    t_sentence = t_sentences[i]
    for j, char in enumerate(x_sentence):
        x_encoder[i, j, char_indices[char]] = 1  # encoderへの入力をone-hot表現で表す
    for j, char in enumerate(t_sentence):
        x_decoder[i, j, char_indices[char]] = 1  # decoderへの入力をone-hot表現で表す
        if j > 0:  # 正解は入力より1つ前の時刻のものにする
            t_decoder[i, j-1, char_indices[char]] = 1

print(x_encoder.shape)

In [ ]:
batch_size = 24
epochs = 200
n_mid = 128

In [ ]:
from keras.models import Model
from keras.layers import Dense, GRU, Input, Masking

encoder_input = Input(shape=(None, n_char))
encoder_mask = Masking(mask_value=0)  # 全ての要素が0であるベクトルの入力は無視する
encoder_masked = encoder_mask(encoder_input)
encoder_lstm = GRU(n_mid, dropout=0.2, recurrent_dropout=0.2, return_state=True)  # dropoutを設定し、ニューロンをランダムに無効にする
encoder_output, encoder_state_h = encoder_lstm(encoder_masked)

decoder_input = Input(shape=(None, n_char))
decoder_mask = Masking(mask_value=0)  # 全ての要素が0であるベクトルの入力は無視する
decoder_masked = decoder_mask(decoder_input)
decoder_lstm = GRU(n_mid, dropout=0.2, recurrent_dropout=0.2, return_sequences=True, return_state=True)  # dropoutを設定
decoder_output, _ = decoder_lstm(decoder_masked, initial_state=encoder_state_h)  # encoderの状態を初期状態にする
decoder_dense = Dense(n_char, activation='softmax')
decoder_output = decoder_dense(decoder_output)

model = Model([encoder_input, decoder_input], decoder_output)

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
print(model.summary())

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, load_model

early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

history = model.fit([x_encoder, x_decoder], t_decoder,
                     batch_size=batch_size,
                     epochs=epochs,
                     validation_split=0.1,  # 10%は検証用
                     callbacks=[early_stopping])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(np.arange(len(loss)), loss)
plt.plot(np.arange(len(val_loss)), val_loss)
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, load_model
# encoderのモデル
encoder_model = Model(encoder_input, encoder_state_h)

# decoderのモデル
decoder_state_in_h = Input(shape=(n_mid,))
decoder_state_in = [decoder_state_in_h]

decoder_output, decoder_state_h = decoder_lstm(decoder_input,
                                               initial_state=decoder_state_in_h)
decoder_output = decoder_dense(decoder_output)

decoder_model = Model([decoder_input] + decoder_state_in,
                      [decoder_output, decoder_state_h])

# モデルの保存
encoder_model.save('encoder_chumonno_oi_ryouriten_transed.keras')
decoder_model.save('decoder_chumonno_oi_ryouriten_transed.keras')

In [ ]:
print(encoder_model.summary())

In [ ]:
from keras.models import load_model
encoder_model = load_model('encoder_chumonno_oi_ryouriten_transed.keras')
decoder_model = load_model('decoder_chumonno_oi_ryouriten_transed.keras')

def respond(message, beta=5):
    vec = sentence_to_vector(message)  # 文字列をone-hot表現に変換
    print(vec)
    state_value = encoder_model.predict(vec)
    y_decoder = np.zeros((1, 1, n_char))  # decoderの出力を格納する配列
    y_decoder[0][0][char_indices['\t']] = 1  # decoderの最初の入力はタブ。one-hot表現にする。

    respond_sentence = ""  # 返答の文字列
    while True:
        y, h = decoder_model.predict([y_decoder, state_value])
        p_power = y[0][0] ** beta  # 確率分布の調整
        next_index = np.random.choice(len(p_power), p=p_power/np.sum(p_power))
        next_char = indices_char[next_index]  # 次の文字

        if (next_char == "\n" or len(respond_sentence) >= max_length_x):
            break  # 次の文字が改行のとき、もしくは最大文字数を超えたときは終了

        respond_sentence += next_char
        y_decoder = np.zeros((1, 1, n_char))  # 次の時刻の入力
        y_decoder[0][0][next_index] = 1

        state_value = h  # 次の時刻の状態
        print("以下はrespond_sentence")
        print(respond_sentence)
    return respond_sentence

In [ ]:
bot_name = "賢治bot"
your_name = input("おなまえをおしえてください。:")
print()

print(bot_name + ": " + "こんにちは、" + your_name + "さん。")
message = ""
while message != "さようなら。":

    while True:
        message = input(your_name + ": ")
        if not is_invalid(message):
            break
        else:
            print(bot_name + ": ひらがなか、カタカナをつかってください。")

    response = respond(message)
    print(bot_name + ": " + response)